# Get likelihood of priors

## Basic imports

In [ ]:
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats
import scipy.optimize
from tqdm import tqdm

from fair21.energy_balance_model import EnergyBalanceModel

In [ ]:
samples = 1000000

In [ ]:
df_gmst = pd.read_csv('../data/forcing/AR6_GMST.csv')
gmst = df_gmst['gmst'].values

## Load in results

In [ ]:
temp_in = np.load('../data/ar6_ensemble_batches/temperature_1850-2030.npy')
ohc_in = np.load('../data/ar6_ensemble_batches/ohc_2018_minus_1971.npy')
ari_in = np.load('../data/ar6_ensemble_batches/fari_2005-2014_mean.npy')
aci_in = np.load('../data/ar6_ensemble_batches/faci_2005-2014_mean.npy')
co2_in = np.load('../data/ar6_ensemble_batches/co2_2014.npy')
ecs_in = np.load('../data/ar6_ensemble_batches/ecs.npy')
tcr_in = np.load('../data/ar6_ensemble_batches/tcr.npy')
o3_in = np.load('../data/ar6_ensemble_batches/fo3_2019.npy')

In [ ]:
temp_9514_in = np.mean(temp_in[145:165, :], axis=0) - np.mean(temp_in[:51, :], axis=0)

In [ ]:
NINETY_TO_ONESIGMA=scipy.stats.norm.ppf(0.95)

ecs_dist = scipy.stats.skewnorm(8.82185594, loc=1.95059779, scale=1.55584604)
tcr_dist = scipy.stats.norm(loc=1.2, scale=0.6/NINETY_TO_ONESIGMA)
ohc_dist = scipy.stats.norm(loc=434.3730268913012, scale=73.70562981598447)
temp_9514_dist = scipy.stats.skewnorm(-1.65506091, loc=0.92708099, scale=0.12096636)
ari_dist = scipy.stats.norm(loc=-0.3, scale=0.3/NINETY_TO_ONESIGMA)
aci_dist = scipy.stats.norm(loc=-1.0, scale=0.7/NINETY_TO_ONESIGMA)
co2_dist = scipy.stats.norm(loc=397.5469792683919, scale=0.36)
#temp_rmse_dist = (0.5+0.5*np.tanh(8-50*(temp_rmse_out[i, :]))

In [ ]:
pl.plot(np.arange(0, 0.5, 0.01), np.log(0.5+0.5*np.tanh(8-47*(np.arange(0, 0.5, 0.01)))))

In [ ]:
def rmse(obs, mod):
    return np.sqrt(np.sum((obs-mod)**2)/len(obs))

rmse_temp = np.zeros((samples))
for i in tqdm(range(samples)):
    rmse_temp[i] = rmse(gmst[:171], temp_in[:171,i]-temp_in[:51, i].mean())

In [ ]:
log_likelihood = (
    ohc_dist.logpdf(ohc_in/1e21) +
    temp_9514_dist.logpdf(temp_9514_in) +
    ecs_dist.logpdf(ecs_in) +
    tcr_dist.logpdf(tcr_in) +
    ari_dist.logpdf(ari_in) + 
    aci_dist.logpdf(aci_in) +
    co2_dist.logpdf(co2_in) #+
    #np.log(0.5+0.5*np.tanh(8-50*(rmse_temp)))
    np.log(0.5+0.5*np.tanh(8-45*(rmse_temp)))
)

#log_likelihood[np.isinf(log_likelihood)] = 0

In [ ]:
pl.hist(log_likelihood[~np.isinf(log_likelihood)])

In [ ]:
top1000 = np.argpartition(log_likelihood, -1000)[-1000:]

In [ ]:
np.argmax(log_likelihood)

In [ ]:
pl.hist(ecs_in[top1000])
np.percentile(ecs_in[top1000], (5, 16, 50, 84, 95))

In [ ]:
pl.hist(tcr_in[top1000])

In [ ]:
pl.hist(ari_in[top1000])

In [ ]:
pl.hist(aci_in[top1000])
np.percentile(aci_in[top1000], (5, 50,95))

In [ ]:
pl.hist(ohc_in[top1000])

In [ ]:
pl.hist(temp_9514_in[top1000])
np.percentile(temp_9514_in[top1000], (5, 50, 95))

In [ ]:
pl.hist(co2_in[top1000])

In [ ]:
pl.hist(rmse_temp[top1000])

In [ ]:
pl.plot(temp_in[:, top1000]);

In [ ]:
pl.hist(o3_in[top1000])
np.percentile(o3_in[top1000], (5, 16, 50, 84, 95))

In [ ]:
pl.scatter(tcr_in[top1000], aci_in[top1000])